# The progressivity of a UK carbon dividend

Earlier this month, the International Panel on Climate Change ([IPCC](https://www.ipcc.ch/)) released the [third installment](https://report.ipcc.ch/ar6wg3/) of its Sixth Assessment Report, titled "Mitigation of Climate Change." The report covered a wide range of policy options to reduce carbon emissions, as well as a wide range of benefits such policies would have for humanity.

As [Dana Nuccitelli of Citizens' Climate Lobby noted](https://citizensclimatelobby.org/blog/policy/the-latest-ipcc-report-has-a-lot-to-say-about-carbon-fee-and-dividend/), the report has a lot to say about one policy in particular: it contains 680 mentions of carbon pricing, from the economic theory and evidence to policy design and political support. While most climate change proposals require funding, carbon pricing policies raise revenue by charging fossil fuel companies for their carbon pollution. That means it also raises costs in isolation; to address this, the IPCC says, "Carbon pricing is most effective if revenues are redistributed or used impartially." What's more impartial than redistributing to everyone equally?

For Earth Day, we're [following up our report on US carbon dividends](https://www.ubicenter.org/us-carbon-dividend) with an analysis of the policy in the UK. Using [PolicyEngine](http://policyengine.org), we find that levying a fee on each tonne of carbon emissions, and redistributing the proceeds equally to all UK residents, would have a progressive distributional impact. A carbon dividend would significantly reduce poverty and inequality, and the larger the program, the more of each it does.
Methodology

As we've written on the [PolicyEngine blog today](...), PolicyEngine uses a range of data sources and techniques to estimate how carbon taxes would affect different types of households. 


In [6]:
from policyengine.countries import UK
import pandas as pd

uk = UK()

tax_rates = list(range(0, 100, 10))
changes = dict(
    carbon_tax=[],
    ubi=[],
    poverty=[],
    deep_poverty=[],
    child_poverty=[],
    deep_child_poverty=[],
    adult_poverty=[],
    deep_adult_poverty=[],
)

baseline_in_poverty = uk.baseline.calc("in_poverty_bhc", map_to="person")
baseline_in_deep_poverty = uk.baseline.calc("in_deep_poverty_bhc", map_to="person")
is_child = uk.baseline.calc("is_child", map_to="person")
is_adult = uk.baseline.calc("is_adult", map_to="person")

for tax_rate in tax_rates:
    print(tax_rate)
    baseline, reformed = uk._get_microsimulations(dict(carbon_tax=tax_rate))
    revenue = reformed.calc("carbon_tax").sum()
    weekly_ubi = revenue / baseline.calc("people").sum() / 52
    baseline, reformed = uk._get_microsimulations(dict(carbon_tax=tax_rate, child_bi=weekly_ubi, adult_bi=weekly_ubi, senior_bi=weekly_ubi))
    changes["carbon_tax"] += [tax_rate]
    changes["ubi"] += [weekly_ubi]
    in_poverty = reformed.calc("in_poverty_bhc", map_to="person")
    in_deep_poverty = reformed.calc("in_deep_poverty_bhc", map_to="person")
    changes["poverty"] += [in_poverty.mean() / baseline_in_poverty.mean() - 1]
    changes["deep_poverty"] += [in_deep_poverty.mean() / baseline_in_deep_poverty.mean() - 1]
    changes["child_poverty"] += [in_poverty[is_child].mean() / baseline_in_poverty[is_child].mean() - 1]
    changes["deep_child_poverty"] += [in_deep_poverty[is_child].mean() / baseline_in_deep_poverty[is_child].mean() - 1]
    changes["adult_poverty"] += [in_poverty[is_adult].mean() / baseline_in_poverty[is_adult].mean() - 1]
    changes["deep_adult_poverty"] += [in_deep_poverty[is_adult].mean() / baseline_in_deep_poverty[is_adult].mean() - 1]


df = pd.DataFrame(changes)


0
10
20
30
40
50
60
70
80
90


In [7]:
import plotly.express as px
from ubicenter import format_fig
from ubicenter.plotly import LIGHT_BLUE

# UBI by tax rate

fig = px.line(df, x="carbon_tax", y="ubi", title="Weekly carbon dividend by per-tonne carbon price", color_discrete_sequence=[LIGHT_BLUE])
fig.update_layout(
    yaxis_tickprefix="£",
    xaxis_tickprefix="£",
    xaxis_title="Carbon price",
    yaxis_title="Weekly dividend",
)
format_fig(fig)

In [9]:
df = df.rename(columns=dict(
    poverty="Poverty",
    deep_poverty="Deep poverty",
    child_poverty="Child poverty",
    deep_child_poverty="Deep child poverty",
    adult_poverty="Adult poverty",
    deep_adult_poverty="Deep adult poverty",
    carbon_tax="Carbon price"
))
from ubicenter.plotly import DARK_BLUE
fig = px.line(
    df,
    x="Carbon price",
    y=[
        "Poverty",
        "Deep poverty",
    ],
    color_discrete_sequence=[LIGHT_BLUE, DARK_BLUE],
).update_layout(
    title="Carbon dividend poverty impact by carbon price",
    yaxis_tickformat=".0%",
    xaxis_tickprefix="£",
    legend_title="",
    yaxis_title="Poverty impact",
)
format_fig(fig)

In [12]:
# Current the dataframe has columns [price, ubi, poverty, deep poverty, child poverty, deep child poverty, working-age poverty, deep working-age poverty, senior poverty, deep senior poverty]
# We need it with columns [price, age group, poverty type, value]

long_df = pd.melt(df, id_vars=["Carbon price"], value_vars=[
    "Poverty",
    "Deep poverty",
    "Child poverty",
    "Deep child poverty",
    "Adult poverty",
    "Deep adult poverty",
])
long_df["Age group"] = long_df.variable.map({
    "Poverty": "All",
    "Deep poverty": "All",
    "Child poverty": "Children",
    "Deep child poverty": "Children",
    "Adult poverty": "Adults",
    "Deep adult poverty": "Adults",
})
long_df["Poverty type"] = long_df.variable.apply(lambda x: "Deep poverty" if "deep" in x.lower() else "Poverty")
import plotly.io as pio
pio.renderers.default = "notebook_connected" # As per https://github.com/microsoft/vscode-jupyter/issues/4364#issuecomment-817352686
fig = px.bar(
    long_df,
    x="Age group",
    y="value",
    color="Poverty type",
    barmode="group",
    animation_frame="Carbon price",
    color_discrete_sequence=[LIGHT_BLUE, DARK_BLUE],
).update_layout(
    yaxis_title="Poverty impact",
    yaxis_tickformat=".0%",
    title="Carbon dividend poverty impact by age group",
    legend_title="",
    yaxis_range=[-0.4, 0],
)
format_fig(fig)